### Globals

In [30]:
import os
import glob
import subprocess
import sys

rootdir = '/project_freenas/3022017.02/UKB'
qcdir = os.path.join(rootdir, 'qc', 'dMRI')

if os.path.exists(qcdir) == False:
    os.mkdir(os.path.join(rootdir, 'qc', 'dMRI'))
else: pass

cluster_log_dir = '/home/preclineu/ramcir/DCCN/TorqueLogs'

cmd_qsub_base = ['/home/preclineu/ramcir/DCCN/Scripts/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '2gb',
                 '-logfiledir', cluster_log_dir
                ]

# get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(rootdir,'subjects','*')))
print('Found', len(sub_dirs), 'subjects to process')

Found 42766 subjects to process


### Sort subjects

In [47]:
sub_no_dmri = []
sub_no_eddy = []
sub_ready = []
sub_complete = []

# Control the number of subjects
sdirs = sub_dirs[0:19]

for s in sdirs:
    subid = os.path.splitext(os.path.basename(s))[0]

    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    quaddir = os.path.join(dmridir, 'data.qc')  
    eddyfile = os.path.join(dmridir, 'data.eddy_parameters')
    
    try:
        #Check if dMRI data is avalable
        if os.path.exists(dmridir) == False:  
            print(subid,' dMRI data not found.')
            sub_no_dmri.append(subid)
        else: 
            #Check if EDDY files are available
            if os.path.exists(eddyfile) == False:
                print(subid,' EDDY data not found.')
                sub_no_eddy.append(subid)
            else:  
                #Check if quad has already been run
                if os.path.exists(quaddir) == True:
                    print(subid,' QC already done.') 
                    sub_complete.append(s)
                else:
                    print(subid,' is ready for QC') 
                    sub_ready.append(s)
    except:
        print(subid, ' ignored due to', sys.exc_info()[0], '. Continuing with the next subject.')

1000050  EDDY data not found.
1000432  QC already done.
1000853  QC already done.
1000871  QC already done.
1000910  QC already done.
1001018  dMRI data not found.
1001169  dMRI data not found.
1001197  EDDY data not found.
1001309  QC already done.
1001410  EDDY data not found.
1001525  QC already done.
1001679  QC already done.
1001815  QC already done.
1001849  dMRI data not found.
1001922  QC already done.
1001958  QC already done.
1002081  is ready for QC
1002118  is ready for QC
1002421  is ready for QC


### Run QUAD

In [33]:
quad_paths = os.path.join(qcdir,'quad_paths.txt')
open(quad_paths, 'a').close()

In [34]:
for s in sub_ready[0:10]:  
    subid = os.path.splitext(os.path.basename(s))[0]
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    mask_dir = os.path.join(dmridir,'nodif_mask.nii.gz')
    acqparam_file = os.path.join(dmridir, 'acqparam.txt')
    idx_file = os.path.join(dmridir, 'index.txt')
    quaddir = os.path.join(dmridir, 'data.qc')  

    
    if os.path.exists(quaddir) == True:
        print(subid,' QC already done. Doing nothing')
    else:
        print(subid,' QUAD running')
        #Create symbolic link for data_ud with the name data
        #This is necessary for running 'quad' since all other eddy outputs are called data.eddy.
        os.chdir(dmridir)
        if os.path.islink('./data.nii.gz') == False:
            os.symlink('./data_ud.nii.gz', './data.nii.gz')
        else: pass

        #Create aquision parameters file and index file
        #The acquisition parameters file is based on dataset acq documentation
        #i.e. AP dir = 0 1 0, PA dir = 0 -1 0, Echo spacing = 0.67 ms, EPI factor = 104 (for UKB)
        if os.path.exists(acqparam_file) == False:
            with open(os.path.join(dmridir,'acqparams.txt'), 'w') as acqparams:
                acqparams.write('0 1 0 0.069\n0 -1 0 0.069')
        else:pass

        #The index file indicated the direction in which each volume was aquired (based on aqcparams.txt)
        #count number of volumes present in data
        if os.path.exists(idx_file) == False:
            with open(os.path.join(dmridir,'bvals'), 'r') as bvals:
                data = bvals.read()
                vols = data.split()
                no_vols = len(vols)
            with open(os.path.join(dmridir,'index.txt'), 'w') as index:
                for vol in range(0,no_vols):    
                    index.write('1 ')
        else:pass
    
        #Write command for creating the b0 mask and run the bash process
        if os.path.exists(mask_dir) == False:
            fslroi_cmd = ['fslroi ' + dmridir + '/data.nii.gz',
                           dmridir + '/b0_data.nii.gz' ' 0 1']
            fslbet_cmd = ['bet ' + dmridir + '/b0_data.nii.gz',
                           dmridir + '/nodif' ' -f 0.1 -g 0 -m -n']
      
        else:pass
    
        #Write command for running QUAD      
        quad_cmd = ['eddy_quad ' + dmridir + '/data',
                     '-idx ' + dmridir + '/index.txt',
                     '-par ' + dmridir + '/acqparams.txt',
                     '-m ' + dmridir + '/nodif_mask', 
                     '-b ' + dmridir + '/bvals ' 
                     '-g '  + dmridir + '/bvecs']

        quad_cmd_str = '"%s"' % str(' '.join(quad_cmd)) 

        #Add the path of the qc folder to txt file (will be needed for SQUAD)
        append_cmd = ['echo -e "', quaddir, '" >> ', quad_paths ]
        append_cmd_str = '%s' % str(''.join(append_cmd))

        #Join comands and sumbit process
        proc_cmd = '"%s"' % str(' '.join(fslroi_cmd) + ' && ' + ' '.join(fslbet_cmd) + ' && ' + ' '.join(quad_cmd) + ' && ' + ''.join(append_cmd))
        cmd_qsub = cmd_qsub_base + ['-name', 'UKB_quad','-command', proc_cmd]     
        subprocess.run(' '.join(cmd_qsub), shell=True)
             

1000432  QUAD running
1000853  QUAD running
1000871  QUAD running
1000910  QUAD running
1001309  QUAD running
1001525  QUAD running
1001679  QUAD running
1001815  QUAD running
1001922  QUAD running
1001958  QUAD running


### Run SQUAD
Only run after all subjects have quad

In [39]:
squad_cmd = 'eddy_squad'
quad_folders = os.path.join(qcdir,'quad_paths.txt')
prep_cmd = squad_cmd + ' ' + os.path.join(qcdir,'quad_paths.txt') + ' -o ' + os.path.join(qcdir,'squad')
print(prep_cmd)
subprocess.run(prep_cmd, shell = True)        

eddy_squad /project_freenas/3022017.02/UKB/qc/dMRI/quad_paths.txt -o /project_freenas/3022017.02/UKB/qc/dMRI/squad


CompletedProcess(args='eddy_squad /project_freenas/3022017.02/UKB/qc/dMRI/quad_paths.txt -o /project_freenas/3022017.02/UKB/qc/dMRI/squad', returncode=0)

### Run slicesdir
To check the binary mask

In [50]:
string = ""

#this is where the slicesdir will be saved
os.chdir(qcdir)

#append the path to b0_data and nodif_mask for each sub
for s in sub_complete[0:9]:
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    string+=str(dmridir + '/b0_data.nii.gz ' + dmridir + '/nodif_mask.nii.gz' + ' ')

#prep and run the slicesdir command
slices_cmd = 'slicesdir'
prep_cmd = slices_cmd + ' -o ' + string
subprocess.run(prep_cmd, shell = True)

CompletedProcess(args='slicesdir -o /project_freenas/3022017.02/UKB/subjects/1000432/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1000432/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1000853/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1000853/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1000871/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1000871/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1000910/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1000910/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1001309/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1001309/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1001525/dMRI/dMRI/b0_data.nii.gz /project_freenas/3022017.02/UKB/subjects/1001525/dMRI/dMRI/nodif_mask.nii.gz /project_freenas/3022017.02/UKB/subjects/1001679/dMRI/dMRI